In [61]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.corpus import stopwords
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from wordcloud import WordCloud
import gensim
# from keras import layers
# from keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB


In [36]:

data = pd.read_csv('train.csv/train.csv')



data['is_toxic'] = np.where((data['toxic'] + data['severe_toxic'] + data['obscene']+ data['threat'] + data['insult'] + data['identity_hate'])!= 0, 1, 0)
# data.head()
# data[data['is_toxic'] == 1].head()

In [109]:
data = data.loc[((data['comment_text'].str.contains('{', regex = False) - 1)* -1).astype(bool)]

In [39]:
# IMBALANCED LEARN STUFF
from imblearn.under_sampling import ClusterCentroids, RandomUnderSampler, NearMiss

In [111]:
from sklearn.model_selection import KFold
from collections import defaultdict
from sklearn.metrics import classification_report, accuracy_score, f1_score, recall_score
def undersample(x, y, method = RandomUnderSampler(random_state = 1), return_all = True):
    '''
    x: x data
    y: y data
    method: undersampling method to be used
    return_all: if we want to return the y of the undersampled as well
    
    returns: Undersampled x, <<undersampled y if return_all>>
    '''
    if return_all:
        return method.fit_resample(x,y)
    return method.fit_resample(x,y)[0]

def transform_to_useable(x):
    '''
    x: x that still the comment text as well as possibly other catiggories
    
    returns: vecotrized comment text, df of columns in x other then comment text
    '''
    tfd = TfidfVectorizer(stop_words='english',max_features=5000)
    X = tfd.fit_transform(x['comment_text'])
    y = x.loc[:, x.columns != 'comment_text']
    return X, y


def k_fold_val(X, y, models = [LogisticRegression(C = 12, max_iter = 500)], folds = 5):
    '''
    X: the x of the data
    y: the result of the data
    models: the models that we want tested
    folds: number of kfolds to do
    
    return: {(model, catigory_modeled): {
        'report': [classification report from kfolds]
        'acc': [acc scores from kfolds]
        'f1': [f1 score from kfolds]
        'recall': [recall from kfolds]
    }}
    '''
    kf = KFold(n_splits = folds)
    kf.get_n_splits(X)
    results = defaultdict(list)
    for model in models:
        for cat in y:
            print(f'doing kfold for {model} on {cat}')
            for i, (train, test) in enumerate(kf.split(X)):
                y_sub = y[cat]
                model.fit(X[train], y_sub[train])
                # might want to add in train results
                y_pred = model.predict(X[test])

                results[(model, cat)].append({'report': classification_report(y_sub[test], y_pred),
                                             'acc':accuracy_score(y_sub[test], y_pred),
                                             'f1': f1_score(y_sub[test], y_pred),
                                             'recall': recall_score(y_sub[test], y_pred),
                                             })
    return results

def full_run(x, y, undersample_method = RandomUnderSampler(random_state = 1), combined_x_y = True, models = [LogisticRegression(C = 12, max_iter = 500)], folds = 5):
    '''
    ALL BEFORE UNDERSAMPLING
    X: the x data before vectorization
    y: the labels of data
    
    undersample_method: the way to undersample, might be able to use oversample as well? need to check
    combined_x_y: Is the data for Y inside of X, did you create a seperate classification to do undersampling
    models: the models to be tested
    folds: number of of kfolds to test
    
    return: {(model, catigory_modeled): {
        'report': [classification report from kfolds]
        'acc': [acc scores from kfolds]
        'f1': [f1 score from kfolds]
        'recall': [recall from kfolds]
    }}
    '''
    
    if combined_x_y:
        X = undersample(x, y, undersample_method, return_all = False)
        X, y = transform_to_useable(X)
    else:
        X, y = undersample(x, y, undersample_method, return_all = True)
        X, _ = transform_to_useable(X)
    y = y.loc[:, y.columns != 'id']
    return k_fold_val(X, y, models, folds)

def view_report(report, info = 'report', to_consol = True):
    '''
    report: the report produced by k_fold_val or full_run
    info: the statstic that you want to see about the validations
    to_consol: if you want it printed
    
    TODO: add a return to method that helps us pick model
    '''
    for key, value in reports.items():
        print(key)
        for v in value:
            print(v[info])
        print('---------------------------------------------------------------')

In [114]:
X_before_vec = data.loc[:, data.columns != 'is_toxic'] 
y_sampling = data.loc[:, data.columns == 'is_toxic']
sample = RandomUnderSampler(random_state = 1)
models = [LogisticRegression(C = 12, max_iter = 500)]
reports = full_run(X_before_vec, y_sampling, undersample_method = sample, models = models)


doing kfold for LogisticRegression(C=12, max_iter=500) on toxic


C:\Users\reada\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\reada\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\reada\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetri

doing kfold for LogisticRegression(C=12, max_iter=500) on severe_toxic


C:\Users\reada\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
C:\Users\reada\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\reada\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1609: UndefinedMetricWarning: 

doing kfold for LogisticRegression(C=12, max_iter=500) on obscene


...

C:\Users\reada\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
C:\Users\reada\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\reada\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: 

doing kfold for LogisticRegression(C=12, max_iter=500) on insult


C:\Users\reada\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\reada\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\reada\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetri

doing kfold for LogisticRegression(C=12, max_iter=500) on identity_hate


C:\Users\reada\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\reada\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\reada\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetri

In [115]:
view_report(reports, 'recall')

(LogisticRegression(C=12, max_iter=500), 'toxic')
0.0
0.0
0.8422774869109948
0.7973770491803279
0.7971419185282523
---------------------------------------------------------------
(LogisticRegression(C=12, max_iter=500), 'severe_toxic')
0.0
0.0
0.2953846153846154
0.25990491283676703
0.2911392405063291
---------------------------------------------------------------
(LogisticRegression(C=12, max_iter=500), 'obscene')
0.0
0.0
0.7413479052823315
0.7166324435318275
0.7283546563522761
---------------------------------------------------------------
(LogisticRegression(C=12, max_iter=500), 'threat')
0.0
0.0
0.21782178217821782
0.23267326732673269
0.3063583815028902
---------------------------------------------------------------
(LogisticRegression(C=12, max_iter=500), 'insult')
0.0
0.0
0.6181584030907921
0.5989895800442059
0.6131503351420364
---------------------------------------------------------------
(LogisticRegression(C=12, max_iter=500), 'identity_hate')
0.0
0.0
0.33212996389891697
0.286